# PS 2
### Zhou Xing

In [9]:
import timeit
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,ShuffleSplit, cross_val_score
from sklearn.metrics import accuracy_score
from dask.distributed import Client
from dask import delayed
import multiprocessing
from dask import compute, delayed
import dask.multiprocessing
import warnings
import multiprocessing
from sklearn import preprocessing

## Question 1

In [2]:
auto = pd.read_csv("data/Auto.csv", na_values="?")
auto.dropna(inplace=True)
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [4]:
auto["mpg_high"] = np.where(auto["mpg"] > auto["mpg"].median(), 1, 0)
auto = pd.get_dummies(auto, prefix='origin', columns=['origin']).drop(columns=['origin_3'])
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,name,mpg_high,origin_1,origin_2
0,18.0,8,307.0,130.0,3504,12.0,70,chevrolet chevelle malibu,0,1,0
1,15.0,8,350.0,165.0,3693,11.5,70,buick skylark 320,0,1,0
2,18.0,8,318.0,150.0,3436,11.0,70,plymouth satellite,0,1,0
3,16.0,8,304.0,150.0,3433,12.0,70,amc rebel sst,0,1,0
4,17.0,8,302.0,140.0,3449,10.5,70,ford torino,0,1,0


In [12]:
X = auto_df[['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin_1', 'origin_2']].values
X = preprocessing.scale(X)
y = auto[['mpg_high']].values.reshape(len(X), )

In [15]:
%%time
# Set the cross-validation method
rs = ShuffleSplit(n_splits=100, train_size=.65, random_state=0)

# Set the Logistic regressor
logit_reg = LogisticRegression(n_jobs=1)

# Cross validate and get the score
scores = cross_val_score(logit_reg, X, y, cv=rs, scoring='accuracy')

# Calculate the mean error rate
print('Mean error rate: ', 1 - scores.mean())

Mean error rate:  0.0952898550724639
CPU times: user 131 ms, sys: 9.94 ms, total: 141 ms
Wall time: 136 ms


/Users/zhou/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/zhou/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/zhou/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/zhou/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/zhou/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 

### Problem 2

In [18]:
num_cores = multiprocessing.cpu_count()
print('# available cores is', num_cores)

# available cores is 12


In [33]:
@delayed
def cal_accuracy(bootstrap, seed, X, y): 
    X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=.65, random_state=seed)
    model = LogisticRegression(solver="lbfgs", max_iter=200, n_jobs=1).fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = ((y_test - y_pred) ** 2).mean()
    return mse

In [34]:
error_rate_list = []

for i in range(100): 
    error_rate_list.append(cal_accuracy(i, 0, X, y))
    
mean_error_rate = delayed(lambda x: sum(x)/100)(error_rate_list)

In [35]:
%%time
print('Mean error rate: ', mean_error_rate.compute())

Mean error rate:  0.11594202898550728
CPU times: user 425 ms, sys: 64 ms, total: 488 ms
Wall time: 432 ms
